In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# load folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# please change the path of your dataset here below
ROOT_PATH = "/content/drive/MyDrive/colab notebook/data_exp/german_med_termss/"
# ROOT_PATH= "updated_german_med_terms/"

In [ ]:
# # install all needed libraries
# !pip install transformers
# !pip install pytorch-lightning
# !pip install transformers datasets --quiet
# !pip install -U imbalanced-learn
# !pip install scikit-multilearn

# !pip install spacy
# !pip install spacy-transformers
# !python3 -m spacy download de_dep_news_trf
# !python3 -m spacy download de_core_news_lg

# # !python -m spacy download de_dep_news_trf

!pip install transformers
!pip install pytorch-lightning
!pip install transformers datasets --quiet
!pip install -U imbalanced-learn
!pip install scikit-multilearn

In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification, BertTokenizer, BertForMultipleChoice
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import ast
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
import datasets
from datasets import Dataset

import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification, BertTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import ast
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
import datasets
from datasets import Dataset

In [ ]:
files = os.listdir(ROOT_PATH)
filepaths = [ROOT_PATH + f for f in os.listdir(ROOT_PATH) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df.shape

In [ ]:
def tx(x):
  res = []
  for v in ast.literal_eval(x):
    res.append(str(v).strip().lower())
  return res


df['labels_array']= df['expertise'].apply(lambda x: tx(x))
df = df[pd.notna(df['labels_array'])]
df.shape

In [ ]:
def get_categories_with_index(labels_classfified: dict):
    categories = []
    main = labels_classfified['Unnamed: 1']
    for k,v in main.items():
        if type(v) == str:
            categories.append(v)
    
    categories_with_index = []
    for i in range(len(categories) -1 ):
        start = list(main.values()).index(str(categories[i]))
        end = list(main.values()).index(str(categories[i + 1])) - 1

        main_category = str(categories[i]).lower()
        categories_with_index.append({"category": categories[i], "start": start, "end": end, "values": [main_category]})
    
    # for the last category
    last_category = categories[-1].lower()
    categories_with_index.append({"category": last_category, "start": -1, "end": -1, "values": [last_category]})
    
    columns = list(labels_classfified.keys())[2:]
    for v in columns:
        for ci in categories_with_index:
            ci['category'] = str(ci['category']).lower()
            ci['values'] += found(labels_classfified[v], ci)
        
    return categories_with_index

def found(dict_under, ci):
    values = []
    for k,v in dict_under.items():
        if k > ci['start'] and k < ci['end'] and type(v) == str:
            values += [str(v).lower()]
    
    return list(set(values))

categories_with_index = get_categories_with_index(pd.read_excel('../content/drive/MyDrive/colab notebook/Praxisprojekt_MMC.xlsx').to_dict())
# categories_with_index = get_categories_with_index(pd.read_excel('Praxisprojekt_MMC.xlsx').to_dict())
target_names = [c['category'] for c in categories_with_index]
print(target_names)

In [ ]:
def change_label_to_main_category(x):
    result = []
    for value_from_x in x:
        for ci in categories_with_index:
            if value_from_x in ci['values']:
                result.append(ci['category'])

    return list(set(result))

df['labels_array']= df['labels_array'].apply(lambda x: change_label_to_main_category(x))

In [ ]:
print(df['labels_array'].value_counts())

In [ ]:

def get_number_of_sample_per_label(df, number_of_sample_per_label):
    result = []

    for label, count in df['labels_array'].value_counts().items():
        if count <= number_of_sample_per_label:
            result += [label]

    print("count -> ", len(result))
    return result

found_labels = get_number_of_sample_per_label(df, 1)
print(found_labels)

if len(found_labels) > 0:
    ri = []
    for index, row in df.iterrows():
        for l in found_labels:
            if l == row['labels_array']:
                ri.append(index)

    print("ri", ri)
    df.drop(index=ri, inplace = True)
    df.shape

In [ ]:
print(df['labels_array'].value_counts())

In [ ]:
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels_array'])
print(df_train.shape, df_val.shape)

In [ ]:
def explode_doc(df):
    df['doc'] = df[['all_page', 'wiki_content']].apply(lambda x: [str(x['all_page']) ]+ [str(x['wiki_content']) ], axis=1)
    df = df.explode("doc")

    df = df[pd.notna(df['doc'])]
    df['doc'] = df['doc'].replace('nan', np.nan)
    df.dropna(subset=['doc'], inplace=True)  

    return df

def explode_labels_array(df):

    df = df.explode("labels_array")
    df.rename(columns = {'labels_array': 'label'}, inplace = True)

    df['label'].replace('', np.nan, inplace=True)
    df.dropna(subset=['label'], inplace=True) 

    sns.set(rc={'figure.figsize':(11.7,8.27)})
    sns.countplot(data=df, y="label", orient="v")

    return df

def get_label_encoder(df):
    label_encoder = LabelEncoder()
    df['label'] = label_encoder.fit_transform(df['label'])
    # df['label'] = df['label'].apply(str)

    le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print(le_name_mapping)
    
    return df

In [ ]:
# print("---------------------------LABELS ARRAYS--------------------------------")
# print(df_train['labels_array'].value_counts())
# print("-----------------------------------------------------------------")
# print(df_val['labels_array'].value_counts())


df_train = get_label_encoder(explode_labels_array(explode_doc(df_train)))
df_val = get_label_encoder(explode_labels_array(explode_doc(df_val)))

print(df_train.shape)
print(df_val.shape)

# print("---------------------------LABELS--------------------------------")
# print(df_train['label'].value_counts())
# print("-----------------------------------------------------------------")
# print(df_val['label'].value_counts())

In [ ]:
###################################################################
# df_train = df_train.head(10)
# df_val = df_val.head(2)
####################################################################

In [ ]:
models = [
    "emilyalsentzer/Bio_ClinicalBERT", # 
    "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", # 
    "bert-base-uncased", # epoch = 2 --> F1 = 0.5346
    "fspanda/Medical-Bio-BERT2", #  "----------------------------"
    "bert-base-multilingual-cased", #
    "bert-base-german-cased", #
    "smanjil/German-MedBERT", #
    "deutsche-telekom/bert-multi-english-german-squad2", #
]


PRE_TRAINED_MODEL_NAME = models[5] # models[6] need 25 epochs for training
print(PRE_TRAINED_MODEL_NAME)

MAX_LEN = 512
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
train_encodings = tokenizer(df_train['doc'].tolist(), truncation=True, padding=True, max_length=MAX_LEN)
valid_encodings = tokenizer(df_val['doc'].tolist(), truncation=True, padding=True, max_length=MAX_LEN)

In [ ]:
class ExpDataset(torch.utils.data.Dataset):
    def __init__(self, docs, labels):
        self.docs = docs
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.docs.items()}
        item["labels"] = torch.tensor([self.labels.values[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = ExpDataset(train_encodings, df_train['label'])
valid_dataset = ExpDataset(valid_encodings, df_val['label'])

In [ ]:
NUM_LABELS = len(list(set(df_train['label'].tolist()))) # len(np.unique(df_train['label'])) neu hunzugefugt

In [ ]:
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=NUM_LABELS).to(device)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)

  print("classification_report --> HERE BELOW")
  print(classification_report(labels, preds))

  f1_score_micro = f1_score(labels, preds, average="micro")
  accuracy = accuracy_score(labels, preds)
  recall = recall_score(labels, preds, average="micro")
  precision = precision_score(labels, preds, average="micro")

  return {'F1_score' : f1_score_micro ,'Accuracy' : accuracy, 'Recall' :recall, 'Precision' : precision}

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,# total number of training epochs
    learning_rate=2e-5,
    #fp16 = True,
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=120,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    save_total_limit = 5,
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    # logging_steps=500,               # log & save weights each logging_steps
    save_strategy = "epoch",
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics     # the callback that computes metrics of interest
)

In [ ]:
# train the model
trainer.train()

In [ ]:
# evaluate the current model after training
trainer.evaluate()

In [ ]:
predictions = trainer.predict(valid_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
trues = [i[0] for i in predictions.label_ids]
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
f,ax = plt.subplots(1,1,figsize=(40,40))
ConfusionMatrixDisplay.from_predictions(trues, preds, ax=ax)

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "pubmed_" + PRE_TRAINED_MODEL_NAME
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)